> T检验及其改进方法

解决问题：

1、转化率

2、分布

3、均值

- 注意：案例部分仅供命令调用使用，与Api部分的输入有差异。

# 转化率

In [3]:
import os
os.chdir('E:\JJabtest') # 切换到一级目录下

In [15]:
import JJabtest

## 模拟数据

In [16]:
from JJabtest.data import exp_data

Data = exp_data.dataab(2,2)  # 对照组、实验组各100个uid，指标值由0和1构成，对照组的batch为1，实验组的batch为2

Data.head(10)

,uid,metric,batch
0,0,1,1
1,1,1,1
2,2,0,1
3,3,1,1
4,4,0,1
5,5,0,1
6,6,0,1
7,7,1,1
8,8,1,1
9,9,1,1


In [17]:
from JJabtest.model.ttest import t_test

data = Data[['metric','batch']]

result = t_test(data,batcha = 1,batchb = 2,alpha = 0.05,input_power=0.80,ttest_plus_method = False)
result

{'A_metric_avg': 0.5,
 'B_metric_avg': 0.41,
 'lift': -0.18000000000000005,
 'lift_interv_down': -0.45661765213115629,
 'lift_interv_up': 0.096617652131156195,
 'power': 0.24604355998380495,
 'pvalue': 0.20316986304842038}

## 另一种方案

- 上面解决转化率问题时，拼接数据不方便，这里提供一个简单的

输入：

1、对照组的转化人数，对照组的样本UV

2、实验组的转化人数，实验组的样本UV

In [11]:
from JJabtest.model.binomial_conv import bin_conv

Xa = 20  # 对照组转化人数
Xb = 40  # 实验组转化人数
Na = 200 # 对照组UV
Nb = 300 # 实验组UV

bin_conv(Xa,Xb,Na,Nb)

{'avg_a': 0.1,
 'avg_b': 0.13333333333333333,
 'lift': 0.33333333333333326,
 'lift_down': 0.2059484111486414,
 'lift_up': 0.46071825551802509,
 'power': 0.99923477831526075,
 'pvalue': 2.915761506666093e-07}

# 分布

## 模拟数据

In [18]:
from JJabtest.data import exp_data

Data = exp_data.dataab(4,5)  # 对照组、实验组各100个uid，指标值由0和1构成，对照组的batch为1，实验组的batch为2

Data.head(10)

,uid,metric,batch
0,0,1,1
1,1,1,1
2,2,1,1
3,3,1,1
4,4,2,1
5,5,2,1
6,6,0,1
7,7,0,1
8,8,0,1
9,9,1,1


## 指标包含0的情形

In [19]:
from JJabtest.model import t_test

data = Data[['metric','batch']]

result = t_test(data,batcha = 1,batchb = 2,alpha = 0.05,input_power=0.80,ttest_plus_method = False)
result

{'A_metric_avg': 1.45,
 'B_metric_avg': 1.97,
 'lift': 0.35862068965517246,
 'lift_interv_down': 0.12305019248437804,
 'lift_interv_up': 0.59419118682596683,
 'power': 0.84866158420467208,
 'pvalue': 0.0031910227417172384}

## 指标不包含0的情形

In [20]:
import pandas as pd
import numpy as np

def get_dataa(maxa):
    maxa = int(maxa)
    dataa = pd.DataFrame({
            'uid':np.arange(100),
            'metric':np.random.randint(1,maxa,100),
            'batch':np.random.randint(1,2,100)
    },columns=['uid','metric','batch'])

    return dataa

def get_datab(maxb):
    maxb = int(maxb)
    datab = pd.DataFrame({
        'uid': np.arange(100),
        'metric': np.random.randint(2, maxb, 100),
        'batch': np.random.randint(2, 3, 100)
    }, columns=['uid', 'metric','batch'])

    return datab
def dataab(maxa,maxb):
    dataa = get_dataa(maxa)
    datab = get_datab(maxb)
    data = pd.concat([dataa,datab])
    return data

In [21]:
data = dataab(5,5)
data.head(10)

,uid,metric,batch
0,0,4,1
1,1,3,1
2,2,4,1
3,3,1,1
4,4,3,1
5,5,2,1
6,6,3,1
7,7,1,1
8,8,3,1
9,9,3,1


In [22]:
from JJabtest.model.rate_by_event_user import rate_by_event_user

Data = data[['metric','batch']]
result = rate_by_event_user(data=Data,batch_a=1,batch_b=2,alpha=0.05,input_power=0.80)
result

{'A_metric_avg': 2.28,
 'B_metric_avg': 3.0,
 'lift': 0.3157894736842106,
 'lift_interv_down': 0.19900800377107003,
 'lift_interv_up': 0.43257094359735121,
 'power': 0.99961289511934026,
 'pvalue': 3.1350504443556682e-07}

# 均值

## 模拟数据

In [23]:
from JJabtest.data import exp_data

Data = exp_data.dataab(4,5)  # 对照组、实验组各100个uid，指标值由0和1构成，对照组的batch为1，实验组的batch为2

Data.head(10)

,uid,metric,batch
0,0,1,1
1,1,0,1
2,2,1,1
3,3,0,1
4,4,0,1
5,5,1,1
6,6,0,1
7,7,2,1
8,8,0,1
9,9,3,1


## 指标中包含0的情形

In [24]:
from JJabtest.model.ttest import t_test
data = Data[['metric','batch']]
result = t_test(data=data, batcha=1, batchb=2, alpha=0.05, input_power=0.80, ttest_plus_method=True)

result 

{'A_metric_avg': 1.57,
 'B_metric_avg': 1.83,
 'lift': 0.16560509554140126,
 'lift_interv_down': -0.051531390671505327,
 'lift_interv_up': 0.38274158175430789,
 'power': 0.32008145766385848,
 'pvalue': 0.13607356742293283}

## 指标不包含0的情形

In [25]:
data = dataab(5,6)
data.head(10)

,uid,metric,batch
0,0,1,1
1,1,4,1
2,2,2,1
3,3,2,1
4,4,4,1
5,5,3,1
6,6,3,1
7,7,4,1
8,8,2,1
9,9,3,1


In [26]:
from JJabtest.model.rate_by_event import rate_by_event

data = data[['metric','batch']]
result = rate_by_event(data=data, batch_a=1, batch_b=2, alpha=0.05, input_power=0.80)

result

{'A_metric_avg': 2.41,
 'B_metric_avg': 3.51,
 'lift': 0.45643153526970937,
 'lift_interv_down': 0.33041242013975197,
 'lift_interv_up': 0.5824506503996667,
 'power': 0.99999988327938616,
 'pvalue': 2.1081985481649537e-11}